In [246]:
import requests
import cssutils
from bs4 import BeautifulSoup
import os
import json
import pandas as pd
import numpy as np

base_link = "https://www.comics.org"
headers = {'cookie': 'csrftoken=QHJodRDYKJURtwTstbtNgm11Y9CQXpah9bcrpeEQzidXymRdveyB7v40kxX9BBod; _pbjs_userid_consent_data=3524755945110770; SL_G_WPT_TO=tr; SL_GWPT_Show_Hide_tmp=1; SL_wptGlobTipTmp=1; na-unifiedid=%7B%22TDID_LOOKUP%22%3A%22FALSE%22%2C%22TDID_CREATED_AT%22%3A%222022-10-25T16%3A13%3A58%22%7D; __cf_bm=rhAa8ngQuuaZS_oCQmF0SUmhwjb7_UmRg4S.3700sEE-1666716009-0-AR/tYj4CL2rJORh5yxy5tEaYkgOQelY5ZdFtZ7P3TuL4w0i5BmpH0qA3yvu+KqgxDZtcIjZvGL/LRrDQyspWsstClokAEzoLDkm7lJrr+q3CSj1DZvWppNa7FL5Ugcp2uQ==',
          'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.52'}
marvel_id = 78

In [247]:
def get_page_number(publisher_id):
    link = f'https://www.comics.org/publisher/{publisher_id}'
    req = requests.get(link,  headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    page_bar_div = soup.find("div", {"class":"right pagination"})
    page_bar_div_li = page_bar_div.find_all("a", {"class": "btn btn-default btn-sm"})
    
    page_number = int(page_bar_div_li[-2]['href'].split("=")[-1])
    
    return page_number


In [248]:
def scrap_comic_list(page_number):
    datas = []
    for idx in range(page_number):
        link = f'https://www.comics.org/publisher/78/?sort=year&page={idx+1}'
        
        req = requests.get(link, headers=headers)
        soup = BeautifulSoup(req.text, 'html.parser')
        
        comic_table = soup.find("table", {"class":"sortable_listing"})
        
        #Comic list columns name
        columns_name = soup.find("thead").find_all("th")
        columns_name = [i.text for i in columns_name]
        
        #Comic list data
        data = soup.find("tbody").find_all("tr")
        for data_piece in data:
            data_info = data_piece.find_all("td")
            data_info_list = [i.text for i in data_info]
            
            #Comic list link data
            link_list = [base_link + i['href'] for i in data_piece.find_all("a")]
            
            datas.append(data_info_list+link_list)
        
        return columns_name + ['Series Link', 'Covers1 Link', 'Covers2 Link'], datas
        break


In [249]:
marvel_page_number = get_page_number(marvel_id)

In [250]:
columns_name, datas = scrap_comic_list(marvel_page_number)

In [253]:
df_comic_list = pd.DataFrame(datas, columns=columns_name).replace({None: np.nan})

In [256]:
df_comic_list.to_csv("./data/comic_list.csv", index=False)